In [ ]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import requests

CATALOG_URL = "http://server:8181/catalog"

SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = ".".join(SPARK_VERSION.split(".")[:2])
ICEBERG_VERSION = "1.8.1"

In [ ]:
requests.post(
    "http://server:8181/management/v1/warehouse",
    json={
        "warehouse-name": "hdfs",
        "storage-profile": {
            "type": "hdfs",
            "key-prefix": "/user/hdfs",
            "url": "hdfs://namenode:8020",
        },
    },
).content

# Connect with Spark

In [ ]:
config = {
    "spark.sql.catalog.lakekeeper": "org.apache.iceberg.spark.SparkCatalog",
    "spark.sql.catalog.lakekeeper.type": "rest",
    "spark.sql.catalog.lakekeeper.uri": CATALOG_URL,
    "spark.sql.catalog.lakekeeper.warehouse": "hdfs",
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.defaultCatalog": "lakekeeper",
    "spark.jars.packages": f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}",
}

In [ ]:
spark_config = SparkConf().setMaster("local").setAppName("Iceberg-REST")
for k, v in config.items():
    spark_config = spark_config.set(k, v)

spark = SparkSession.builder.config(conf=spark_config).getOrCreate()
spark.sparkContext.setLogLevel("DEBUG")

spark.sql("USE lakekeeper")

## Read and Write Tables

In [ ]:
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS my_namespace")
spark.sql("SHOW NAMESPACES").toPandas()

In [ ]:
data = pd.DataFrame([[1, "a-string", 2.2]], columns=["id", "strings", "floats"])
sdf = spark.createDataFrame(data)

In [ ]:
sdf.writeTo(f"my_namespace.my_table").createOrReplace()

In [ ]:
spark.sql(f"SELECT * FROM my_namespace.my_table").toPandas()